In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/vgg19"))
print(os.listdir("../input/resnet50"))
print(os.listdir("../input/resnet50trainedwithaptosolddataset"))
print(os.listdir("../input"))
print(os.listdir('../input/dr-data-processed/'))
print(os.listdir('../input/pretrained-weights-vgg19'))

# Any results you write to the current directory are saved as output.

**0. Combine data**

In [ ]:
import matplotlib.pyplot as plt

data_19_dir = '../input/dr-data-processed/'

train_15_class_true = pd.read_csv(os.path.join(data_19_dir, 'labels/trainLabels15.csv'))
train_15_class_true.rename(columns={'image':'id_code', 'level':'diagnosis'}, inplace=True)
train_15_class_true["id_code"]=train_15_class_true["id_code"].apply(lambda x: "train_15_processed/" + x + ".jpg")
print(train_15_class_true.head())
plt.figure(1)
plt.hist(train_15_class_true['diagnosis'], bins=5)
plt.title('train 15 distribution')

test_15_class_true = pd.read_csv(os.path.join(data_19_dir, 'labels/testLabels15.csv'))
test_15_class_true.rename(columns={'image':'id_code', 'level':'diagnosis'}, inplace=True)
test_15_class_true["id_code"]=test_15_class_true["id_code"].apply(lambda x: "test_15_processed/" + x + ".jpg")
test_15_class_true = test_15_class_true.drop(columns=['Usage'])
print(test_15_class_true.head())
plt.figure(2)
plt.hist(test_15_class_true['diagnosis'], bins=5)
plt.title('test 15 distribution')

from sklearn.utils import shuffle

pretrain_class_true = pd.concat([train_15_class_true, test_15_class_true], axis=0, ignore_index=True)
pretrain_class_true = shuffle(pretrain_class_true)
pretrain_class_true = pretrain_class_true.reset_index(drop=True)
class_count = pretrain_class_true['diagnosis'].value_counts()
pretrain_class_weight = {c: np.sqrt(np.max(class_count)/class_count[c]) for c in [0, 1, 2, 3, 4]}
print(pretrain_class_weight)
pretrain_class_true['diagnosis'] = pretrain_class_true['diagnosis'].astype(str)

**1. Preprocess images**

In [ ]:
import random
import math
import cv2

global C, L
C = 5
L = 224

# Preprocessing functions
"""
remove the black borders of an img
"""
def cut_black(img, tol=5):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = img_gray > tol
    idx = np.ix_(mask.any(1),mask.any(0))
    return img[idx[0], idx[1], :]


"""
Only take the center
"""
def crop_center(img):
    H, W = img.shape[0], img.shape[1]
    if H == W:
        return img
    elif H > W:
        return img[H//2-W//2:H//2+W//2, :, :]
    else:
        return img[:, W//2-H//2:W//2+H//2, :]
        
        
"""
Adjust brightness, scale to mean 100
"""
def adjust_light(img):
    brightness = np.mean(img)
    img = np.clip(100.0/brightness*img, 0, 255).astype('uint8')
    return img


"""
Crop image based off black pixels on diagon
"""
def crop_diagonal(img, tol=5):
    img_diag = np.diagonal(img)
    img_diag_gray = np.mean(img_diag, axis=0).astype('int32')
    idx0 = np.argmax(img_diag_gray>tol)
    idx1 = len(img_diag_gray) - np.argmax(img_diag_gray[::-1]>tol)
    return img[idx0:idx1, idx0:idx1, :]


"""
Subtract median blur
"""
def subtract_median_bg_image(img):
    k = np.max(img.shape)//20*2+1
    bg = cv2.medianBlur(img, k)
    return cv2.addWeighted(img, 4, bg, -4, 128)


"""
Full preprocessing steps
"""
def img_preprocess(img):
    img = cut_black(img)
    img = crop_center(img)
    img = crop_diagonal(img)
    #img = adjust_light(img)
    img = subtract_median_bg_image(img)
    img = cv2.resize(img, dsize=(L, L), interpolation=cv2.INTER_CUBIC)
    return img

In [ ]:
data_dir = "../input/aptos2019-blindness-detection/"
train_dir = os.path.join(data_dir, "train_images")
test_dir = os.path.join(data_dir, "test_images")
train_dir_processed = "../input/train_processed"
test_dir_processed = "../input/test_processed"
train_label_file = os.path.join(data_dir, "train.csv")
test_name_file = os.path.join(data_dir, "test.csv")

if not os.path.exists(train_dir_processed):
    os.mkdir(train_dir_processed)
if not os.path.exists(test_dir_processed):
    os.mkdir(test_dir_processed)

train_labels = pd.read_csv(train_label_file)
classes = train_labels['diagnosis'].unique()

In [ ]:
train_imgs = os.listdir(train_dir)
N_train = len(train_imgs)
print("process train, n = " + str(N_train))
os.system('echo ' + 'process train, n = ' + str(N_train))
for train_img in train_imgs:
    img = cv2.imread(os.path.join(train_dir, train_img))
    img = img_preprocess(img)
    cv2.imwrite(os.path.join(train_dir_processed, train_img), img)

test_imgs = os.listdir(test_dir)
N_test = len(test_imgs)
print("process test, n = " + str(N_test))
os.system('echo ' + 'process test, n = ' + str(N_test))
for test_img in test_imgs:
    img = cv2.imread(os.path.join(test_dir, test_img))
    img = img_preprocess(img)
    cv2.imwrite(os.path.join(test_dir_processed, test_img), img)

**2. Organize training & validation data for fit_generator**

In [ ]:
from sklearn.utils import shuffle

train_class_true = pd.read_csv(train_label_file)
train_class_true["id_code"]=train_class_true["id_code"].apply(lambda x: x + ".png")
train_class_true = shuffle(train_class_true)
train_class_true = train_class_true.reset_index(drop=True)
train_class_true['diagnosis'] = train_class_true['diagnosis'].astype(str)
class_count = train_class_true['diagnosis'].value_counts()
class_count = class_count.sort_index()
class_weight = {c: np.sqrt(np.max(class_count)/class_count[c]) for c in [0, 1, 2, 3, 4]}
print(class_weight)

**3. Build model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

"""
Single conv layer in CNN model
"""
def conv_layer(x_in, filters, kernel_dim, drop_rate=0.0, batch_norm=True, max_pool=True):
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(kernel_dim,kernel_dim), strides=(1,1), padding='same',
               kernel_initializer='he_normal')(x_in)
    x = tf.keras.layers.Activation('relu')(x)
    if drop_rate > 0.0:
        x = tf.keras.layers.Dropout(drop_rate)(x)
    if batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)
    if max_pool:
        x = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
    return x
    

"""
Single dense/FC layer in CNN model
"""
def dense_layer(x_in, units, activation='tanh', drop_rate=0.0):
    x = tf.keras.layers.Dense(units, activation=activation)(x_in)
    if drop_rate > 0.0:
        x = tf.keras.layers.Dropout(rate = drop_rate)(x)
    return x

"""
Custom loss function
"""
def custom_loss(y_true, y_pred):
    class_true = K.cast(K.expand_dims(K.argmax(y_true, axis=-1), axis=-1), 'float32')   # y_true is one-hot
    i = np.array([[0, 1, 2, 3, 4]]).astype('float32')
    alpha = 3 * 30 * K.square(i - class_true) / K.sum(K.square(i - class_true), axis=0)
    # cross entropy
    loss1 = -K.sum(y_true * K.log(y_pred), axis=-1)
    # additional term to penalize worse predictions
    loss2 = -K.sum(alpha * (1-y_true) * K.log(1-y_pred), axis=-1)
    
    return loss1 + loss2


"""
Custom eval metric: quadratic weighted kappa
"""
def qwk(y_true, y_pred):
    # compute confution matrix
    y_true_label = K.argmax(y_true, axis=-1)    # one-hot to class number
    y_pred_label = K.argmax(y_pred, axis=-1)
    confusion = tf.math.confusion_matrix(y_true_label, y_pred_label, num_classes=5, dtype='float32')
    
    # compute quadratic weight
    alpha = np.square([[i-j for i in range(5)] for j in range(5)]).astype('float32')

    # compute observed and expected matrix
    observed = confusion/tf.reduce_sum(confusion)  # count -> distribution
    P_pred = tf.reduce_sum(confusion, axis=0)/tf.reduce_sum(confusion)
    P_true = tf.reduce_sum(confusion, axis=1)/tf.reduce_sum(confusion)
    expected = tf.tensordot(P_true, P_pred, axes=0)
    
    # compute kappa
    kappa = 1 - tf.reduce_sum(tf.multiply(alpha, observed))/tf.reduce_sum(tf.multiply(alpha, expected))
    return kappa


"""
Use transfer learning
"""
def transfer_model(loss, lr=1e-3):
    #base_model = tf.keras.applications.VGG16(input_shape=(L,L,3),
    #                                         weights='../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
    #                                         include_top=False)
    
    base_model = tf.keras.applications.VGG19(input_shape=(L,L,3),
                                             weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                             include_top=False)
    
    #base_model = tf.keras.applications.ResNet50(input_shape=(L,L,3),
    #                                            weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    #                                            include_top=False)

    #base_model = tf.keras.applications.ResNet50(input_shape=(L,L,3),
    #                                            weights='../input/resnet50trainedwithaptosolddataset/Resnet50_bestqwk.h5',
    #                                            include_top=True)
    
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    
    # dense layers
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)    # faster processing, omit spacial, vessel leakage can happen anywhere
    #x = tf.keras.layers.GlobalMaxPooling2D()(x)         # you only need to find certain features once, other areas can be blank
    #x = dense_layer(x, units=1024, activation='elu', drop_rate=0.4)
    x = dense_layer(x, units=256, activation='tanh', drop_rate=0.4)
    
    # class pred
    x_out = tf.keras.layers.Dense(5, activation='softmax')(x)
    
    # compile model
    model = tf.keras.models.Model(inputs=base_model.input, outputs=x_out)
    #sgd = tf.keras.optimizers.SGD(learning_rate=1e-1)
    adam = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    
    if loss == 'categorical_crossentropy':
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc', qwk])
    else:
        model.compile(optimizer=adam, loss=custom_loss, metrics=['acc', qwk])
    
    return model

In [ ]:
import matplotlib.pyplot as plt

"""
Display training history
"""
def show_history(history):
    acc=history.history['acc']
    val_acc=history.history['val_acc']
    qwk=history.history['qwk']
    val_qwk=history.history['val_qwk']
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    epochs = list(range(1, len(acc)+1))
    plt.figure(1)
    plt.plot(epochs, loss, 'b', epochs, val_loss, 'r')
    plt.title('loss')
    plt.legend(('train', 'val'))
    
    plt.figure(2)
    plt.plot(epochs, acc, 'b', epochs, val_acc, 'r')
    plt.title('accuracy')
    plt.legend(('train', 'val'))
    
    plt.figure(3)
    plt.plot(epochs, qwk, 'b', epochs, val_qwk, 'r')
    plt.title('weighted kappa')
    plt.legend(('train', 'val'))

    plt.figure(4)
    plt.plot(loss, qwk)
    plt.title('train: qwk vs loss')
    
    plt.figure(5)
    plt.plot(val_loss, val_qwk)
    plt.title('val: qwk vs loss')

**4. Generate datasets**

In [ ]:
batch_size = 128

print("Train set:")
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   #rotation_range=360,  # an eyeball at any angle is still the same
                                   #horizontal_flip=True,        # left/right eye symmetry
                                   #vertical_flip=True,          # left/right eye symmetry
                                   samplewise_center=True,     # centering
                                   samplewise_std_normalization=True, # standardizing
                                   zca_whitening=True,
                                   validation_split=0.1)
    
train_gen = train_datagen.flow_from_dataframe(dataframe=train_class_true,
                                              directory = train_dir_processed,
                                              batch_size=batch_size,
                                              x_col = 'id_code',
                                              y_col = 'diagnosis',
                                              class_mode='categorical',
                                              target_size=(L, L),
                                              subset='training')
    
val_gen = train_datagen.flow_from_dataframe(dataframe=train_class_true,
                                            directory = train_dir_processed,
                                            batch_size=batch_size,
                                            x_col = 'id_code',
                                            y_col = 'diagnosis',
                                            class_mode='categorical',
                                            target_size=(L, L),
                                            subset='validation')

In [ ]:
pretrain_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                      #rotation_range=360,  # an eyeball at any angle is still the same
                                      #horizontal_flip=True,        # left/right eye symmetry
                                      #vertical_flip=True,          # left/right eye symmetry
                                      samplewise_center=True,     # centering
                                      samplewise_std_normalization=True, # standardizing
                                      zca_whitening=True)

pretrain_gen = pretrain_datagen.flow_from_dataframe(dataframe=pretrain_class_true,
                                                 directory = data_19_dir,
                                                 batch_size=128,
                                                 x_col = 'id_code',
                                                 y_col = 'diagnosis',
                                                 class_mode='categorical',
                                                 target_size=(L, L))

**5. Pretrain model on 2015 dataset**

In [ ]:
model = transfer_model(loss = 'categorical_crossentropy')
model.summary()

callbacks1 = [#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',  min_delta=0.0004,
                                                  patience=2, factor=0.5, min_lr=1e-5,  mode='auto', verbose=1),
             tf.keras.callbacks.ModelCheckpoint(filepath = 'pretrained-weights-best-1.hdf5', 
                                                monitor='val_loss', save_best_only=True)]
#             tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 10))]

callbacks2 = [#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',  min_delta=0.0004,
                                                  patience=2, factor=0.5, min_lr=1e-5,  mode='auto', verbose=1),
             tf.keras.callbacks.ModelCheckpoint(filepath = 'pretrained-weights-best-2.hdf5', 
                                                monitor='val_loss', save_best_only=True)]

#os.system('echo ' + 'pretraining')
#history = model.fit_generator(pretrain_gen,
#                              validation_data = val_gen,
#                              steps_per_epoch=math.ceil(pretrain_gen.samples/pretrain_gen.batch_size),
#                              class_weight=pretrain_class_weight,
#                              epochs = 15,
#                              validation_steps=math.ceil(val_gen.samples/val_gen.batch_size),
#                              callbacks = callbacks1,
#                              use_multiprocessing=False,
#                              verbose = 1)
#
#show_history(history)

In [ ]:
os.system('echo ' + 'pretraining 2')
#model = transfer_model(loss = 'custom', lr=1e-4)
#model.load_weights('pretrained-weights-best-1.hdf5')
#model.load_weights('../input/pretrained-weights-vgg16/pretrained-weights-best-1.hdf5')
#history = model.fit_generator(pretrain_gen,
#                              validation_data = val_gen,
#                              steps_per_epoch=math.ceil(pretrain_gen.samples/pretrain_gen.batch_size),
#                              class_weight=pretrain_class_weight,
#                              epochs = 15,
#                              validation_steps=math.ceil(val_gen.samples/val_gen.batch_size),
#                              callbacks = callbacks2,
#                              use_multiprocessing=False,
#                              verbose = 1)

**6. Fine tune on 2019 dataset**

In [ ]:
#model = transfer_model(loss = 'categorical_crossentropy')
#model.summary()
model.load_weights('../input/pretrained-weights-vgg19/pretrained-weights-best-1.hdf5')
#model.load_weights('pretrained-weights-best-2.hdf5')
callbacks1 = [#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',  min_delta=0.0004,
                                                  patience=2, factor=0.5, min_lr=1e-5,  mode='auto', verbose=1),
             tf.keras.callbacks.ModelCheckpoint(filepath = 'weights-best-1.hdf5', 
                                                monitor='val_loss', save_best_only=True)]
#             tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 10))]

callbacks2 = [#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',  min_delta=0.0004,
                                                  patience=2, factor=0.5, min_lr=1e-5,  mode='auto', verbose=1),
             tf.keras.callbacks.ModelCheckpoint(filepath = 'weights-best-2.hdf5', 
                                                monitor='val_loss', save_best_only=True)]

os.system('echo ' + 'training')
history = model.fit_generator(train_gen,
                              validation_data = val_gen,
                              steps_per_epoch=math.ceil(train_gen.samples/train_gen.batch_size),
                              class_weight=class_weight,
                              epochs = 30,
                              validation_steps=math.ceil(val_gen.samples/val_gen.batch_size),
                              callbacks = callbacks1,
                              use_multiprocessing=False,
                              verbose = 1)

show_history(history)

In [ ]:
os.system('echo ' + 'training 2')
model = transfer_model(loss = 'custom', lr=1e-4)
model.load_weights('weights-best-1.hdf5')
#model.load_weights('pretrained-weights-best-2.hdf5')
history = model.fit_generator(train_gen,
                              validation_data = val_gen,
                              steps_per_epoch=math.ceil(train_gen.samples/train_gen.batch_size),
                              class_weight=class_weight,
                              epochs = 40,
                              validation_steps=math.ceil(val_gen.samples/val_gen.batch_size),
                              callbacks = callbacks2,
                              use_multiprocessing=False,
                              verbose = 1)

show_history(history)

model.load_weights('weights-best-2.hdf5')
val_pred = model.predict_generator(val_gen, workers=1)    # workers!=1 will mess up the order
val_pred_class = np.argmax(val_pred, axis=1)
plt.figure(6)
plt.hist(val_pred_class, bins=5)
plt.title('val pred distribution')

**5. Generate test set results**

test_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  zca_whitening=True)
test_gen = test_datagen.flow_from_dataframe(  
        dataframe=test_15_class_true,
        directory = test,    
        x_col="id_code",
        target_size = (L, L),
        #preprocessing_function=img_preprocess,
        batch_size = 1,
        shuffle = False,
        class_mode = None
        )

test_pred = model.predict_generator(test_gen, workers=1)    # workers!=1 will mess up the order
test_pred_class = np.argmax(test_pred, axis=1)
plt.figure(7)
plt.hist(test_pred_class, bins=5)
plt.title('test pred distribution')

In [ ]:
output_filename = 'submission.csv'

# copied from a kernel
test_class_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_class_df["id_code"] = test_class_df["id_code"].apply(lambda x: x + ".png")

test_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  zca_whitening=True)
test_gen = test_datagen.flow_from_dataframe(  
        dataframe=test_class_df,
        directory = test_dir_processed,    
        x_col="id_code",
        target_size = (L, L),
        #preprocessing_function=img_preprocess,
        batch_size = 1,
        shuffle = False,
        class_mode = None
        )

test_gen.reset()

predict = model.predict_generator(test_gen,
                                  steps = len(test_gen.filenames),
                                  use_multiprocessing=False)

filenames = test_gen.filenames
results = pd.DataFrame({"id_code":filenames,
                        "diagnosis":np.argmax(predict,axis=1)})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results.to_csv(output_filename, index=False)

In [ ]:
test_pred_class = pd.read_csv(output_filename)
#print(test_pred_class.head())
plt.hist(test_pred_class['diagnosis'], bins=5)
plt.title('test pred distribution')